In [1]:
from google.cloud import storage
from google.oauth2 import service_account
from more_itertools import bucket
from pandera import Column, Check
from schema import Schema, And, Use, Optional, SchemaError
import json
import os
import io
import pandas as pd
import pandera as pa

with open('..\security\parameters.json', 'r') as json_file:
    parameters = json.load(json_file)

bucket_name = parameters['bucket_name']

credentials = service_account.Credentials.from_service_account_file("..\security\parameters-key.json")
storage_client = storage.Client(credentials=credentials)
bucket = storage_client.bucket(bucket_name)

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from schema import Schema, And, Use

schema_hired_employees1 = Schema({
    'id': And(Use(int)),
    'name': And(Use(str)),
    'datetime': And(Use(str)),
    'de_id': And(Use(int)),
    'job_id': And(Use(int))
})

schema_departments1 = Schema({
    'id': And(Use(int)),
    'department': And(Use(str))
})

schema_jobs1 = Schema({
    'id': And(Use(int)),
    'job': And(Use(str))
})


In [3]:
columns_hired_employees = ['id', 'name', 'datetime', 'departament_id', 'job_id']
columns_jobs = ['id', 'job']
columns_departments = ['id', 'department']

In [4]:
def validate_dataframe(filename, columns_names, schema_table, chunksize = 1000):
    #Variables 
    data_validate = pd.DataFrame(columns=columns_names)
    data_error = pd.DataFrame(columns=columns_names)
    indexs_errors = pd.Series([],dtype=pd.Int32Dtype()) 

    #Data Blob
    blob = bucket.blob(filename, chunk_size = 256 * 1024)
    data = blob.download_as_string()
   
    for chunk in pd.read_csv(io.BytesIO(data), names=columns_names, header=None, chunksize=chunksize):
        try:
            schema_table(chunk, lazy=True)
        except pa.errors.SchemaErrors as exc:
            indexs_errors = pd.concat([indexs_errors, exc.failure_cases["index"]])
            print('Cantidad de errores: ', len(indexs_errors.index))
        data_validate = pd.concat([data_validate, chunk[~chunk.index.isin(indexs_errors)]])
        data_error = pd.concat([data_error, chunk[chunk.index.isin(indexs_errors)]])

    return data_validate, data_error

In [6]:
blob2 = bucket.blob('poc-ingest/jobs.csv', chunk_size = 256 * 1024)
data2 = blob2.download_as_string()
#print(data)

In [8]:
filename = "..\\files_csv\\hired_employees.csv"

chunksize = 1000
for chunk in pd.read_csv(filename, chunksize=chunksize):
    #print(chunk.count)
    print(chunk.size)

5000
4990
